# cuPauliProp 예제 - Pauli Propagation 완전 가이드

## 개요
**목표**: Pauli propagation을 사용한 기댓값 계산 완전 이해

**핵심 이론**: Heisenberg picture에서 $\langle 0|U^\dagger O U|0\rangle$ 계산

**학습 순서**:
1. Setup: 라이브러리 및 기본 파라미터
2. Observable 초기화: Pauli string 인코딩
3. Buffer 준비: GPU 메모리 할당
4. Circuit 구성: Gate operators
5. Propagation: Observable 변환
6. 계산: Trace 계산
7. 검증: 상태 벡터 비교
8. 정리: 리소스 해제

---

## 핵심 개념

**Pauli 인코딩**:
- X mask: X 또는 Y가 있는 큐비트 위치
- Z mask: Z 또는 Y가 있는 큐비트 위치
- 예: Z₀ = (X=0b00, Z=0b01), Y₀ = (X=0b01, Z=0b01)

**Trace 계산**:
- ⟨0|I|0⟩ = 1 (항등원소)
- ⟨0|X|0⟩ = 0 (직교)
- ⟨0|Y|0⟩ = 0 (직교)
- ⟨0|Z|0⟩ = 1 (고유상태)

**Gate 종류**:
- Pauli Rotation: RX(θ), RY(θ), RZ(θ)
- Clifford: H, X, Y, Z, S, S†, CNOT 등

---
## Phase 1: Setup
### Step 1: 라이브러리 임포트 및 Handle 생성

In [1]:
import numpy as np
import cupy as cp
from cuquantum.bindings import cupauliprop as cupp

print("✓ cuPauliProp 라이브러리 로드 완료")
print(f"  NumPy: {np.__version__}")
print(f"  CuPy: {cp.__version__}")

✓ cuPauliProp 라이브러리 로드 완료
  NumPy: 2.4.1
  CuPy: 13.6.0


작업 엔진 생성

In [2]:
# 기존 handle이 있으면 정리 (Run All 반복 실행 대비)
try:
    if 'handle' in globals():
        try:
            cupp.destroy(handle)
        except:
            pass
except:
    pass

handle = cupp.create()
print("✓ cuPauliProp handle 생성 완료")

✓ cuPauliProp handle 생성 완료


### Step 2: 시스템 파라미터 설정

2큐비트 시스템에 대한 기본 설정:

**Packed Integers**: Pauli 연산자를 비트로 인코딩하기 위한 64비트 정수 개수
- 2 qubits → 1개의 uint64로 충분
- 각 큐비트는 X mask와 Z mask로 표현됨

In [3]:
num_qubits = 8
num_packed_ints = cupp.get_num_packed_integers(num_qubits)

print(f"큐비트 수: {num_qubits}")
print(f"Packed integers: {num_packed_ints}")

큐비트 수: 8
Packed integers: 1


---
## Phase 2: Observable 초기화
### Step 3: 초기 Observable 설정 (Z₀)

**Pauli 연산자 인코딩**:
- Pauli string은 X mask와 Z mask로 표현
- X mask: X 또는 Y 연산자가 있는 큐비트 위치
- Z mask: Z 또는 Y 연산자가 있는 큐비트 위치

**예시 (단일 항)**:
- Z₀: X_mask=0b00, Z_mask=0b01 → `[0b00, 0b01]`
- Z₁: X_mask=0b00, Z_mask=0b10 → `[0b00, 0b10]`
- X₀: X_mask=0b01, Z_mask=0b00 → `[0b01, 0b00]`
- Y₀: X_mask=0b01, Z_mask=0b01 → `[0b01, 0b01]`

**여러 항의 선형 합**:

API 문서에 따르면, 여러 Pauli 항은 다음 순서로 저장:
```python
# ZZZ + XYI 예시 (API 문서):
# 항1(ZZZ): X_mask=000, Z_mask=111
# 항2(XYI): X_mask=110, Z_mask=010
# → [000, 111, 110, 010]
# 즉, [항1_X, 항1_Z, 항2_X, 항2_Z, ...]
```

**데이터 구조 (정확한 형식)**:
```python
# 단일 항: [X_mask, Z_mask]
d_input_pauli = cp.array([0b00, 0b01], dtype=cp.uint64)  # Z₀

# 여러 항 (예: Z₀ + X₁):
d_input_pauli = cp.array([
    0b00, 0b01,  # 항1: Z₀ [X_mask, Z_mask]
    0b10, 0b00   # 항2: X₁ [X_mask, Z_mask]
], dtype=cp.uint64)
d_input_coef = cp.array([1.0, 1.0], dtype=cp.float64)
```

In [4]:
# Z₀Z₁Z₂Z₃ observable 생성
# Pauli 데이터 구조: [X_mask, Z_mask] 순서

# d_input_pauli = cp.array([0b00, 0b10000000], dtype=cp.uint64) # 15큐
# d_input_coef = cp.array([1.0], dtype=cp.float64)

# print("✓ Observable 설정: Z₀Z₁Z₂Z₃")
# print(f"  X mask: {bin(int(d_input_pauli[0]))}")  # 0b0000
# print(f"  Z mask: {bin(int(d_input_pauli[1]))}")  # 0b1111
# print(f"  계수: {float(d_input_coef[0])}")

# 다른 예시들 (주석):
# d_input_pauli = cp.array([0b10, 0b01], dtype=cp.uint64)  # Z₀X₁
# d_input_pauli = cp.array([0b00, 0b10], dtype=cp.uint64)  # Z₁

# 여러 항의 선형 합 예시:
# d_input_pauli = cp.array([
#     0b01, 0b00,  # Z₀: [X_mask, Z_mask]
#     0b00, 0b10   # X₁: [X_mask, Z_mask]
# ], dtype=cp.uint64)
# d_input_coef = cp.array([1.0, 0.3], dtype=cp.float64)  # 1.0×Z₀ + 0.3×X₁


In [5]:
# # 10-qubit fixed Hamiltonian (6 terms, mixed X/Y/Z)
num_qubits = 10
num_packed_ints = cupp.get_num_packed_integers(num_qubits)  # =1 for 10 qubits

# 각 항의 (X_mask, Z_mask) — 비트는 큐비트 인덱스와 동일 (LSB가 qubit 0)
terms = [
    # coef,   X_mask (bin),    Z_mask (bin)      # Pauli string 설명
    ( 0.8,    0b0000000101,    0b0000000010),    # X0 X2 Z1
    (-0.6,    0b0000010000,    0b0000001000),    # X4 Z3
    ( 0.5,    0b0000000000,    0b0000010011),    # Z0 Z1 Z4
    ( 0.4,    0b0000001100,    0b0000001100),    # Y2 Y3  (X=1,Z=1 → Y)
   # (-0.3,    0b0001000000,    0b0000100000),    # X6 Z5
   # ( 0.9,    0b0100000000,    0b1000000000),    # X9 Z8
]


# # interleave [X0, Z0, X1, Z1, ...]
pauli_list = []
coef_list = []
for c, xm, zm in terms:
    pauli_list.extend([xm, zm])
    coef_list.append(c)

d_input_pauli = cp.array(pauli_list, dtype=cp.uint64)
d_input_coef  = cp.array(coef_list, dtype=cp.float64)

print("✓ Fixed 10-qubit Hamiltonian built")
for i, (c, xm, zm) in enumerate(terms):
    print(f"Term {i}: X={xm:010b} Z={zm:010b} coef={c:+.3f}")

✓ Fixed 10-qubit Hamiltonian built
Term 0: X=0000000101 Z=0000000010 coef=+0.800
Term 1: X=0000010000 Z=0000001000 coef=-0.600
Term 2: X=0000000000 Z=0000010011 coef=+0.500
Term 3: X=0000001100 Z=0000001100 coef=+0.400


### Step 4: 비트 순서 확인 (중요!)

API 문서에 따르면: **n번 큐비트는 n번 비트에 대응**되며, `(packedint >> n) & 1`로 접근합니다.

즉, 비트는 **오른쪽부터** 0, 1, 2, ... 순서입니다 (Little-endian 스타일).

---
## Phase 3: Buffer 및 Expansion 준비
### Step 5: Pauli Expansion 생성

Pauli expansion은 GPU에서 Pauli 연산자를 처리하기 위한 자료구조입니다.

**중요**: Non-Clifford gate (예: RY, RZ)는 Pauli term을 분기시켜 2배씩 증가시킵니다.
- 초기: Z₀ (1개 term)
- RY 적용 후: 여러 Pauli term으로 확장 가능
- 충분한 `max_terms`를 설정해야 합니다

In [6]:
max_terms = 150000          # Layer 3+ 대비 버퍼 확장 (50000 → 200000)
# ⚠️ max_terms = 게이트 적용 AFTER 최대 항 개수 (버퍼 용량)
# 
# ❌ NOT "초기 관측자의 최대 항 개수"
# ✅ YES "Rotation 게이트로 분기된 항들까지 포함해서 버퍼에 저장 가능한 최대 항 개수"
#
# 이유:
# - Clifford: 1 → 1 (분기 없음)
# - Pauli rotation: 1 → 최대 2 (분기됨)
#
# 예시 (초기 1개 항):
# - Rotation 1개: 최대 2개 항
# - Rotation 2개: 최대 4개 항
# - Rotation N개: 최대 2^N개 항
#
# 따라서 회로에 Rotation이 N개면 max_terms ≥ 2^N 이어야 함

########################################################################################
# GPU 메모리에 저장공간 확보 및 초기 데이터 복사
########################################################################################
pauli_buffer_size = 2 * num_packed_ints * max_terms * 8  # bytes
coef_buffer_size = max_terms * 8  # bytes (float64)

# Input/Output buffers
d_input_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=cp.uint64)
d_input_coef_buffer = cp.zeros(max_terms, dtype=cp.float64)
d_output_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=cp.uint64)
d_output_coef_buffer = cp.zeros(max_terms, dtype=cp.float64)

# 초기 데이터 복사
d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli  
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

def reset_expansions():
    """입력/출력 expansion을 원본 observable로 재설정."""
    global input_expansion, output_expansion
    # 버퍼 초기화
    d_input_pauli_buffer.fill(0)
    d_input_coef_buffer.fill(0)
    d_output_pauli_buffer.fill(0)
    d_output_coef_buffer.fill(0)
    # 원본 observable 재적재
    d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli
    d_input_coef_buffer[:len(d_input_coef)] = d_input_coef
    # 기존 expansion 제거 후 재생성
    try:
        if 'input_expansion' in globals():
            cupp.destroy_pauli_expansion(input_expansion)
        if 'output_expansion' in globals():
            cupp.destroy_pauli_expansion(output_expansion)
    except Exception:
        pass
    input_expansion = cupp.create_pauli_expansion(
        handle,
        num_qubits,
        d_input_pauli_buffer.data.ptr,
        pauli_buffer_size,
        d_input_coef_buffer.data.ptr,
        coef_buffer_size,
        1,
        len(d_input_coef),
        1,
        1,
    )
    output_expansion = cupp.create_pauli_expansion(
        handle,
        num_qubits,
        d_output_pauli_buffer.data.ptr,
        pauli_buffer_size,
        d_output_coef_buffer.data.ptr,
        coef_buffer_size,
        1,
        0,
        0,
        0,
    )

# 최초 1회 생성
reset_expansions()

print(f"✓ Input/Output expansion 준비 완료 (capacity: {max_terms} terms, initial: {len(d_input_coef)} terms)")########################################################################################

✓ Input/Output expansion 준비 완료 (capacity: 150000 terms, initial: 4 terms)


### Step 6: Workspace 준비

Workspace는 GPU 계산 중 필요한 임시 메모리입니다.

**필요한 이유**:
- Non-Clifford gate는 Pauli term을 분기시킴
- 중간 계산 결과를 GPU 메모리에 임시 저장
- 대략적인 크기: `circuit_depth × max_terms × 1 MB`

---

## 💡 Workspace 이해하기

**Workspace가 필요한 이유**:
1. **중간 계산 결과 저장**: Gate 연산 시 임시 데이터 저장
2. **Term 분기 처리**: Rotation gate가 1개 term을 2개로 분기할 때 임시 공간 필요
3. **정렬/병합**: 중복 term 제거 시 임시 버퍼

**Workspace 크기 계산**:
```
workspace_size ≈ max_terms × num_packed_ints × 64 bytes × safety_factor
```

**Layer 증가 시 문제**:
- Layer 2: 24 rotations → 최악의 경우 2^24 = 16M terms (실제는 훨씬 적음)
- Layer 3: 36 rotations → 최악의 경우 2^36 = 68B terms (불가능!)
- **해결책**: 중간에 sort=True로 중복 제거 필요

**최적화 전략**:
1. **주기적 정리**: 매 layer 후 중복 term 병합
2. **Threshold 설정**: 작은 계수 제거
3. **Max terms 증가**: GPU 메모리 허용 범위 내에서

In [7]:
# GPU 메모리 할당 전에 기존 할당 정리
if 'workspace' in globals() and workspace is not None:
    try:
        cupp.destroy_workspace_descriptor(workspace)
        print("✓ 이전 workspace 해제됨")
    except:
        pass

# GPU 가용 메모리 확인
mempool = cp.get_default_memory_pool()
free_mem = cp.cuda.Device().mem_info[0]
total_mem = cp.cuda.Device().mem_info[1]
print(f"GPU 가용 메모리: {free_mem / 1024**3:.2f} GB / {total_mem / 1024**3:.2f} GB")

# Workspace 크기 계산
buffer_mem = (d_input_pauli_buffer.nbytes + d_input_coef_buffer.nbytes + 
              d_output_pauli_buffer.nbytes + d_output_coef_buffer.nbytes)
workspace_size = int((free_mem - buffer_mem) * 0.8)

print(f"Buffer 메모리: {buffer_mem / 1024**2:.2f} MB")      
print(f"Workspace 계산: ({free_mem / 1024**3:.2f} GB - {buffer_mem / 1024**2:.2f} MB) × 0.8 = {workspace_size / 1024**3:.2f} GB")

# 새로 할당
d_workspace = cp.cuda.alloc(workspace_size)
workspace = cupp.create_workspace_descriptor(handle)
cupp.workspace_set_memory(
    handle, workspace,
    0,  # CUPAULIPROP_MEMSPACE_DEVICE
    0,  # CUPAULIPROP_WORKSPACE_SCRATCH
    d_workspace.ptr, workspace_size
)
print(f"✓ Workspace allocated")

GPU 가용 메모리: 14.47 GB / 14.58 GB
Buffer 메모리: 6.87 MB
Workspace 계산: (14.47 GB - 6.87 MB) × 0.8 = 11.57 GB
✓ Workspace allocated


---
## Phase 4: Circuit 구성
### Step 7: 양자 회로 구성

**회로**: H(0) → CNOT(0,1) → RY(π/4, qubit 1)

#### Gate 종류

##### 1. Pauli Rotation Gate
- 형태: $e^{-i\theta P/2}$, P는 Pauli 연산자
- API: `create_pauli_rotation_gate_operator`
- Pauli 종류: 0=I, 1=X, 2=Z, 3=Y

##### 2. Clifford Gate
- CNOT, CZ, H, S 등
- API: `create_clifford_gate_operator`
- Gate 종류:
  - 0: I (CNOT)
  - 1: X
  - 2: Z
  - 3: Y
  - 4: H
  - 5: S
  - 7: CX [target, control] 순임;;;
  - 8: CZ
  - 9: CY
  - 10: SWAP
  - 11: ISWAP
  - 12: SQRTX
  - 13: SQRTZ
  - 14: SQRTY

#### Hadamard 분해
H = RZ(π/2) RY(π/2) RZ(π/2)

In [8]:
# 💡 Layer 3+ 실험을 위한 설정
NUM_LAYERS = 3  # 원하는 layer 수로 변경
# TRUNCATION_THRESHOLD = 1e-4  # 이 값보다 작은 계수의 term 제거 (메모리 절약)

print(f"회로 설정:")
print(f"  Layers: {NUM_LAYERS}")
print(f"  Qubits: {num_qubits}")
print(f"  예상 gate 수: {NUM_LAYERS * (3 * num_qubits + (num_qubits-1))}")
print(f"  이론적 최대 terms (중복 제거 전): 2^{NUM_LAYERS * 3 * num_qubits} (실제는 훨씬 적음)")

회로 설정:
  Layers: 3
  Qubits: 10
  예상 gate 수: 117
  이론적 최대 terms (중복 제거 전): 2^90 (실제는 훨씬 적음)


In [9]:
# 기존 gate operators가 있으면 정리 (Run All 반복 실행 대비)
try:
    if 'gate_ops' in globals() and gate_ops:
        for gate in gate_ops:
            try:
                cupp.destroy_operator(gate)
            except:
                pass
except:
    pass

gate_ops = []
PI = np.pi
theta = PI/4

# 포인터 사용 방식
# qubit_0 = np.array([0], dtype=np.int32)
# pauli_y = np.array([3], dtype=np.int32)  # CUPAULIPROP_PAULI_Z (RZ 사용)
# ry = cupp.create_pauli_rotation_gate_operator(
#     handle, theta, 1, qubit_0.ctypes.data, pauli_y.ctypes.data)

# 리스트 전달 방식
for l in range(NUM_LAYERS):
    for q in range(num_qubits):
        rot_x = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [q], [1]) 
        gate_ops.append(rot_x)
        rot_y = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [q], [3]) 
        gate_ops.append(rot_y)
        rot_z = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [q], [2]) 
        gate_ops.append(rot_z)
    for q in range(num_qubits-1):
        cnot = cupp.create_clifford_gate_operator(handle, 7, [q+1, q])  # (타겟, 컨트롤 순임)
        gate_ops.append(cnot)

print(f"✓ {len(gate_ops)}개 gate operators 생성 완료")
    
# ry = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [1], [3]) 
# gate_ops.append(ry)

# np.random.seed(1)  # 재현성 확보
# rand_q = np.random.randint(0, num_qubits, size=20)
# rand_axis = np.random.choice([1, 2, 3], size=20)  # X=1, Z=2, Y=3 (cuPauliProp API 순서)
# rand_degree = np.random.randn(20)
# for i in range(20):
#     qubit = rand_q[i]
#     degree = rand_degree[i]
#     axis = rand_axis[i]
    
#     rot = cupp.create_pauli_rotation_gate_operator(handle, degree, 1, [qubit], [axis])
#     gate_ops.append(rot)



# # 클리포드 게이트#     gate_ops.append(cnot)

# h = cupp.create_clifford_gate_operator(handle, 4, [0]) # H on qubit 0#     cnot = cupp.create_clifford_gate_operator(handle, 7, [i, 0])  # CNOT i→0 (reversed)

# gate_ops.append(h)# for i in range(1, 2):

# CNOT (순서: [control, target])

# ry = cupp.create_pauli_rotation_gate_operator(handle, theta, 1, [1], [3])
# gate_ops.append(ry)

✓ 117개 gate operators 생성 완료


---
## Phase 5: Pauli Propagation 및 계산
### Step 8: Pauli Propagation (Heisenberg Picture)

### 이론

**목표**: $\langle\psi|Z_0|\psi\rangle$ 계산, 여기서 $|\psi\rangle = U|0\rangle$

**Heisenberg picture**:
$$
\langle\psi|Z_0|\psi\rangle = \langle 0|U^\dagger Z_0 U|0\rangle
$$

즉, 상태를 변환하는 대신 Observable을 변환합니다.

### 구현

**중요**: Gate를 역순으로 adjoint 적용
- 회로: $G_1 G_2 G_3$ → U = $G_3 G_2 G_1$
- Propagation: $Z_0$ → $G_3^\dagger Z_0 G_3$ → $G_2^\dagger (G_3^\dagger Z_0 G_3) G_2$ → ...

**API 설정**:
- `adjoint=1`: $G^\dagger O G$ 계산
- 역순 루프: `range(len(circuit)-1, -1, -1)`

In [10]:
import time
print("Pauli propagation (역순, adjoint):")
print("Observable을 circuit의 역방향으로 propagate...\n")

# 리소스 유효성 체크 (리소스 정리 후 재실행 시 대비)
if handle is None or not gate_ops:
    raise RuntimeError(
        "❌ 필요한 리소스가 정리되었습니다.\n"
        "해결 방법: 노트북을 처음부터 다시 실행하세요 (Run All 또는 순차적으로 실행)"
    )

# 매 실행마다 expansion을 원본으로 리셋
reset_expansions()

current_input = input_expansion
current_output = output_expansion

# 역순으로 gate 적용
s = time.time()
for gate_idx, gate in enumerate(reversed(gate_ops)):
    gate_number = len(gate_ops) - gate_idx
    
    # 현재 term 개수 확인
    num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    print(f"Gate {gate_number} (reverse): {num_terms} input terms")
    
    # View 생성
    input_view = cupp.pauli_expansion_get_contiguous_range(
        handle, current_input, 0, num_terms
    )
    
    # Operator 적용: G† O G
    # ⚠️ truncation은 나중에 추가 예정
    cupp.pauli_expansion_view_compute_operator_application(
        handle,
        input_view,
        current_output,
        gate,
        1,  # adjoint=1 → G† O G
        1,  # sort=True → 정렬하여 중복 term 찾기
        0,  # keep_duplicates=False → 중복 term 병합 (계수 합산)
        0,  # num_truncation_strategies=0 (truncation 비활성화)
        None,  # truncation_strategies=None
        workspace  # workspace 포인터 주소
    )
    
    # View 삭제
    cupp.destroy_pauli_expansion_view(input_view)
    
    # Input/Output swap
    current_input, current_output = current_output, current_input
    
    # 진행 상황
    new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    print(f"  → {new_num_terms} output terms")
    
    if new_num_terms >= max_terms:
        raise RuntimeError(
            f"Pauli expansion term overflow: {new_num_terms} >= max_terms={max_terms}. "
            "max_terms를 늘리거나 회로의 rotation 수를 줄이세요.")
    
    # 현재 상태 디버깅: 정확한 buffer 포인터로 접근
    if current_input == input_expansion:
        buffer_pauli = d_input_pauli_buffer
        buffer_coef = d_input_coef_buffer
    else:
        buffer_pauli = d_output_pauli_buffer
        buffer_coef = d_output_coef_buffer
    
    pauli_data = buffer_pauli[:new_num_terms*2*num_packed_ints].get()
    coef_data = buffer_coef[:new_num_terms].get()
    
    for j in range(min(new_num_terms, 5)):  # 최대 5개 항만 출력
        x_mask = pauli_data[2*j]
        z_mask = pauli_data[2*j + 1]
        coef = coef_data[j]
        print(f"    Term {j}: X={x_mask:04b} Z={z_mask:04b} coef={coef:+.4f}")
    if new_num_terms > 5:
        print(f"    ... and {new_num_terms - 5} more terms")
    print()

final_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
print(f"✓ Propagation 완료: 최종 {final_num_terms}개 Pauli terms")
e = time.time()
print(f"총 소요 시간: {e - s:.2f} 초")


Pauli propagation (역순, adjoint):
Observable을 circuit의 역방향으로 propagate...

Gate 117 (reverse): 4 input terms
  → 4 output terms
    Term 0: X=0101 Z=0010 coef=+0.8000
    Term 1: X=10000 Z=1000 coef=-0.6000
    Term 2: X=1100 Z=1100 coef=+0.4000
    Term 3: X=0000 Z=10011 coef=+0.5000

Gate 116 (reverse): 4 input terms
  → 4 output terms
    Term 0: X=0101 Z=0010 coef=+0.8000
    Term 1: X=10000 Z=1000 coef=-0.6000
    Term 2: X=1100 Z=1100 coef=+0.4000
    Term 3: X=0000 Z=10011 coef=+0.5000

Gate 115 (reverse): 4 input terms
  → 4 output terms
    Term 0: X=0101 Z=0010 coef=+0.8000
    Term 1: X=10000 Z=1000 coef=-0.6000
    Term 2: X=1100 Z=1100 coef=+0.4000
    Term 3: X=0000 Z=10011 coef=+0.5000

Gate 114 (reverse): 4 input terms
  → 4 output terms
    Term 0: X=0101 Z=0010 coef=+0.8000
    Term 1: X=10000 Z=1000 coef=-0.6000
    Term 2: X=1100 Z=1100 coef=+0.4000
    Term 3: X=0000 Z=10011 coef=+0.5000

Gate 113 (reverse): 4 input terms
  → 4 output terms
    Term 0: X=0101 Z=0010

### 메모리 절약 전략

**현재 구현된 메모리 절약 방법:**

1. **sort=1, keep_duplicates=0** ✅
   - 동일한 Pauli 항의 계수 자동 합산
   - 중복 제거로 항 개수 감소
   - 별도 설정 불필요

2. **Truncation Strategy** (향후 추가 예정)
   - **COEFFICIENT_BASED**: 계수 크기 < cutoff인 항 제거
   - **PAULI_WEIGHT_BASED**: non-identity Pauli 개수 > cutoff인 항 제거
   - Python 바인딩 제한으로 현재 직접 구현 미지원

**메모리 효과:**
- 중복 제거만: ~20-40% 메모리 절약
- 중복 제거 + truncation (향후): ~60-90% 메모리 절약


### Step 9: 기댓값 계산

**계산**: $\text{Tr}(|0\rangle\langle 0| \cdot observable)$  

이는 $\langle 0\vert observable \vert0\rangle$와 같습니다.

**API**: `pauli_expansion_view_compute_trace_with_zero_state`
- 입력: Propagated observable
- 출력: $\langle 0|O'|0\rangle$

In [11]:
print("기댓값 계산...\n")

# 최종 view 생성
final_view = cupp.pauli_expansion_get_contiguous_range(
    handle, current_input, 0, final_num_terms
)

# 최종 Pauli expansion 내용 디버깅
print("최종 Pauli Expansion 상태:")
if current_input == input_expansion:
    buffer_pauli = d_input_pauli_buffer
    buffer_coef = d_input_coef_buffer
else:
    buffer_pauli = d_output_pauli_buffer
    buffer_coef = d_output_coef_buffer

pauli_data = buffer_pauli[:final_num_terms*2*num_packed_ints].get()
coef_data = buffer_coef[:final_num_terms].get()

for j in range(final_num_terms):
    x_mask = pauli_data[2*j]
    z_mask = pauli_data[2*j + 1]
    coef = coef_data[j]
    print(f"  Term {j}: X={x_mask:04b} Z={z_mask:04b} coef={coef:.6f}")

print()

# Trace 계산: ⟨0|evolved_observable|0⟩
result = np.array([0.0], dtype=np.float64)
cupp.pauli_expansion_view_compute_trace_with_zero_state(
    handle,
    final_view,
    result.ctypes.data,
    workspace  # workspace 포인터 주소
)

expectation_value = result[0]

print("=" * 60)
print(f"결과: ⟨ψ|O|ψ⟩ = {expectation_value:.6f}")
print("=" * 60)


기댓값 계산...

최종 Pauli Expansion 상태:
  Term 0: X=1001 Z=0000 coef=-0.015876
  Term 1: X=1010 Z=0000 coef=0.045185
  Term 2: X=1011 Z=0000 coef=0.037744
  Term 3: X=1100 Z=0000 coef=-0.001105
  Term 4: X=1101 Z=0000 coef=0.002877
  Term 5: X=1110 Z=0000 coef=-0.031898
  Term 6: X=1111 Z=0000 coef=0.018424
  Term 7: X=10000 Z=0000 coef=0.003125
  Term 8: X=10001 Z=0000 coef=-0.007370
  Term 9: X=10010 Z=0000 coef=-0.016624
  Term 10: X=10011 Z=0000 coef=0.001851
  Term 11: X=10100 Z=0000 coef=-0.000129
  Term 12: X=10101 Z=0000 coef=-0.023222
  Term 13: X=10110 Z=0000 coef=0.021571
  Term 14: X=10111 Z=0000 coef=0.007170
  Term 15: X=11000 Z=0000 coef=0.017109
  Term 16: X=11001 Z=0000 coef=0.011502
  Term 17: X=11010 Z=0000 coef=0.001591
  Term 18: X=11011 Z=0000 coef=-0.018043
  Term 19: X=11100 Z=0000 coef=0.000678
  Term 20: X=11101 Z=0000 coef=-0.007693
  Term 21: X=11110 Z=0000 coef=-0.025914
  Term 22: X=11111 Z=0000 coef=0.023808
  Term 23: X=100001 Z=0000 coef=-0.023124
  Term 24: 

In [12]:
# 이상값과 비교를 위한 코드
import numpy as np
import cudaq
from cudaq import spin

# cuPauliProp와 동일한 해밀토니안
H = 0.8*spin.x(0)*spin.x(2)*spin.z(1) - 0.6*spin.x(4)*spin.z(3) + 0.5*spin.z(0)*spin.z(1)*spin.z(4) + 0.4*spin.y(2)*spin.y(3) 
# - 0.3*spin.x(6)*spin.z(5) + 0.9*spin.x(9)*spin.z(8)
# H = spin.x(0) + 0.3*spin.z(1)  # 간단한 예시


@cudaq.kernel
def circuit():
    qv = cudaq.qvector(10)               
    for l in range(3):
        for q in range(10):
            rx(theta, qv[q])
            ry(theta, qv[q])
            rz(theta, qv[q])
        for q in range(9):
            x.ctrl(qv[q], qv[q+1])
        
obs_val = cudaq.observe(circuit, H).expectation()
print(f"cudaq observe expectation = {obs_val:.6f}")

cudaq observe expectation = 0.091256


---
## Phase 6: 리소스 정리
### Step 10: 리소스 정리

모든 생성된 리소스를 올바른 순서로 해제합니다.

In [13]:
print("리소스 정리 중...\n")

# Gate operators 삭제
try:
    if 'gate_ops' in globals():
        for gate in gate_ops:
            try:
                cupp.destroy_operator(gate)
            except:
                pass
        print(f"  ✓ {len(gate_ops)}개 gate operators 삭제")
        gate_ops = []
except Exception as e:
    print(f"  Gate operators 정리 중 에러 (무시): {e}")

# Expansions 삭제
try:
    if 'input_expansion' in globals():
        cupp.destroy_pauli_expansion(input_expansion)
        input_expansion = None
    if 'output_expansion' in globals():
        cupp.destroy_pauli_expansion(output_expansion)
        output_expansion = None
    print("  ✓ Pauli expansions 삭제")
except Exception as e:
    print(f"  Expansions 정리 중 에러 (무시): {e}")

# Workspace 삭제
try:
    if 'workspace' in globals():
        cupp.destroy_workspace_descriptor(workspace)
    print("  ✓ Workspace 삭제")
except Exception as e:
    print(f"  Workspace 정리 중 에러 (무시): {e}")

# Handle 삭제
try:
    if 'handle' in globals():
        cupp.destroy(handle)
        handle = None  # 삭제 후 None으로 설정하여 재실행 시 안전하게
    print("  ✓ Handle 삭제")
except Exception as e:
    print(f"  Handle 정리 중 에러 (무시): {e}")

print("\n✓ 모든 리소스 해제 완료!")

리소스 정리 중...

  ✓ 117개 gate operators 삭제
  ✓ Pauli expansions 삭제
  ✓ Workspace 삭제
  ✓ Handle 삭제

✓ 모든 리소스 해제 완료!


---

## 요약

### 핵심 개념

1. **Packed Integers**: Pauli 연산자를 비트로 인코딩 (X mask, Z mask)
   - 데이터 구조: `[X_mask_0, ..., X_mask_n, Z_mask_0, ..., Z_mask_n]`

2. **Pauli Expansion**: GPU에서 Pauli 연산자 집합을 표현하는 자료구조
   - Input: 초기 observable
   - Output: Gate 적용 결과

3. **Heisenberg Picture**: Observable을 propagate하여 기댓값 계산
   - $\langle\psi|O|\psi\rangle = \langle 0|U^\dagger O U|0\rangle$
   - Gate를 **역순**으로 **adjoint** 적용

4. **Workspace**: GPU 계산용 임시 메모리
   - Non-Clifford gate에서 특히 중요

### 주의사항

- **Pauli 데이터 구조**: 앞쪽 절반이 X mask, 뒤쪽 절반이 Z mask
- **Adjoint 설정**: `adjoint=1`로 설정하여 $G^\dagger O G$ 계산
- **역순 적용**: Circuit을 역순으로 순회
- **충분한 max_terms**: Non-Clifford gate는 term을 2배씩 증가시킴

### 참고 문서

- [cuPauliProp Python API](https://docs.nvidia.com/cuda/cuquantum/latest/python/bindings/cupauliprop.html)